# <center>Circuit Topology script V2.0</center>

<center>Duane Moes - For suggestions and further questions: moesduane@gmail.com </center><br>
<center>Github:   <a href="url" target="https://github.com/Duanetech/circuit_topology">github.com/Duanetech/circuit_topology</a></center>


---
This is a fully automated script that mainly utilizes biopython to perform circuit topology analysis on a given set of proteins. When possible, try to use the mmCIF file system instead of the PDB file option, this is because PDB is outdated and more prone to missing atoms etc.This script serves as an example, you can always create your own loop and use the functions separately if you want to. See the README for installation help and documentation of the functions. If a new update comes out, use the download code option on the github page.

#### Packages used
<ul><li>BioPython</li>
    <li>Pandas</li>
<li>SciPy </li>
<li>NumPy</li>
<li>MatPlotlib</li>
<li>DSSP</li>
</ul>


  
Run the code below to install all the needed dependencies (only once!).<br> Warning: This can take a while, if it has finished you can delete the code block.  

      


In [ ]:
!conda env update --file requirements.yml

#### Importing
These are import statements, you have to run this codeblock everytime you restart and/or quit Jupyter. 

In [1]:
from functions.plots.circuit_plot import circuit_plot
from functions.plots.matrix_plot import matrix_plot
from functions.plots.stats_plot import stats_plot
from functions.plots.matrix_plot_model import matrix_plot_model

from functions.calculating.get_cmap import get_cmap
from functions.calculating.get_matrix import get_matrix
from functions.calculating.get_stats import get_stats
from functions.calculating.energy_cmap import energy_cmap
from functions.calculating.string_pdb import string_pdb
from functions.calculating.secondary_struc_cmap import secondary_struc_cmap
from functions.calculating.secondary_struc_filter import secondary_struc_filter
from functions.calculating.glob_score import glob_score
from functions.calculating.length_filter import length_filter

from functions.importing.retrieve_chain import retrieve_chain
from functions.importing.retrieve_cif import retrieve_cif
from functions.importing.retrieve_cif_list import retrieve_cif_list
from functions.importing.retrieve_secondary_struc import retrieve_secondary_struc
from functions.importing.stride_secondary_struc import stride_secondary_struc

from functions.exporting.export_psc import export_psc
from functions.exporting.export_cmap3 import export_cmap3
from functions.exporting.export_mat import export_mat
from functions.exporting.export_cmap4 import export_cmap4
from functions.exporting.export_circuit import export_circuit

from ipywidgets import widgets
import numpy as np 
import pandas as pd
import os
import matplotlib
%matplotlib

ModuleNotFoundError: No module named 'functions.plots'

## <center> User guide </center>
<ul>
    <li>Either copy your <code>.PDB</code> or <code>.CIF </code> files to their respective maps in <code>/input_files/</code>, or use the retrieve CIF function to download them.
</li>
</ul>

#### Retrieving CIF files
These Functions will automatically download the specified mmCIF files from RCSB PDB to their respective maps in <code>/input_files/</code>. <br>

<i>NOTE that when using a large number of proteins (>50), it is more efficient to use the batch download function from the <a href="url" target="https://www.rcsb.org/downloads">RCSB Db</a> </i><br>

* Use the following function to download a single mmCif file by entering the protein ID <br>

In [3]:
retrieve_cif('1bni')

* The following function Downloads all of the mmCIF files specified in <code>input_files/protlist.txt</code>

In [4]:
retrieve_cif_list()

 
####  ***Variable input*** 
These are variables used for the **Main** script. <br> It is also possible to create your own loop and see the README manual for all the functions.
 
<ul>
<li><code>fileformat</code> Preferred filetype, CIF is recommendend because of a possibility of missing atoms occuring in PDB files. <br></li>
    
  <br>
<li><code>cutoff_distance</code>, maximal distance (Ångström) between two atoms that will count as an atom-atom contact.<br> </li>
<li><code>cutoff_numcontacts</code>, minimum number of contacts between two residues to count as a res-res contact. <br></li>
<li><code>exclude_neighbour</code>, number of neighbours that are excluded from possbile res-res contacts. <br></li>
    <br>
<li><code>length_filtering</code> (0/1), activates length filtering.
<li><code>filtering_distance</code>, specify which max/min distance you want to use.
<li><code>length_mode</code> specifies whether you want short range filtering or long range filtering.
    <br>
    <br>
<li><code>energy_filtering</code>(0/1), activates energy filtering.</li>
 <li><code>energy_filtering_mode</code>(+/-), sets the energy filtering mode.</li>
<br>
<li><code>plot_figures</code>(0/1),Plots figures when activated. Would not recommend with large amound of files.</li> 
<li><code>export_psc</code>(0/1), exporting the resulting PSC stats to a txt file located in <code>results/statistics/psc</code>       (Overwrites a previous created file)</li> 
<li><code>export_cmap3</code>(0/1), exporting the Residue contact map to a csv file located in <code>results/circuit_diagram</code></li> 
<li><code>export_mat</code>(0/1), exporting the topology relations matrix to a csv file located in <code>results/matrix</code></li>
</ul>

In [2]:
# Format
fileformat =            'pdb'

# CT variables
cutoff_distance =       4.5
cutoff_numcontacts =    5
exclude_neighbour =     3

#Length Filtering
length_filtering =      0
filtering_distance =    0
length_mode =           '<'

#Energy filtering
energy_filtering =      0
energy_filtering_mode = '+'

#Time filtering (requires a time filter)
time_filtering = 0

# Exporting
plot_figures =          0
exporting_psc =         1
exporting_cmap3 =       1
exporting_mat   =       0

#### <center>MAIN</center>
-------


##### Single file

In [8]:
#Creates a chain object from a CIF/PDB file
chain,protid = retrieve_chain('1pnj.pdb')

#Step 1 - Draw a residue-residue based contact map 
index,numbering,protid,res_names = get_cmap(
                                            chain,
                                            cutoff_distance = cutoff_distance,
                                            cutoff_numcontacts = cutoff_numcontacts,        
                                            exclude_neighbour = exclude_neighbour)

#Step 3 - Draw a circuit topology relations matrix
mat, psc = get_matrix(index,protid)


FileNotFoundError: [Errno 2] No such file or directory: 'input_files/pdb/1pnj.pdb'

In [15]:
cmap_filter= pd.read_csv('masks/AR/middle_map_run3.csv', header=None)
cmap_filter=np.array(cmap_filter)
cmap_filter.shape

(538, 538)

##### Multiple files

In [3]:
number_of_files = len(os.listdir('input_files/' +fileformat))

psclist = []

for num,files in enumerate(os.listdir('input_files/' +fileformat)):
    if files.endswith(('cif','pdb')):
  
        try:
            #Creates a chain object from a CIF/PDB file
            chain,protid = retrieve_chain(files)
            print(f'{files} - {num+1}/{number_of_files}')
            
        except Exception as e:
            
            print(f'{files} - {e}')
            continue
    else:
        continue
        
    #Step 1 - Draw a residue-residue based contact map 
    index,numbering,protid,res_names = get_cmap(
                                                chain,
                                                cutoff_distance = cutoff_distance,
                                                cutoff_numcontacts = cutoff_numcontacts,        
                                                exclude_neighbour = exclude_neighbour)
    
    #Step 1.5 - Energy filtering
    if energy_filtering:
        index,protid = energy_cmap(index,numbering,
                               res_names,protid,
                               energy_filtering_mode)    
        
    #Step 2 - Lenght filtering
    if length_filtering:
        index = length_filter(index,
                              filtering_distance,
                              length_mode)
        
    if time_filtering:
        cmap_mat = np.zeros([len(numbering),len(numbering)],dtype='int')

        for row in index:
            x = row[0]
            y = row[1]
            cmap_mat[x][y] = 1
            cmap_mat= cmap_mat*cmap_filter
        index=np.array(np.nonzero(cmap_mat))
        index=np.transpose(index)
        
        
    #Step 3 - Draw a circuit topology relations matrix
    mat, psc = get_matrix(index,protid)
    psclist.append(psc)
    
    #Step 4 - Circuit topology statistics
    entangled = get_stats(mat)
    
    #Plotting
    if plot_figures:
        circuit_plot(index,protid,numbering)
        matrix_plot(mat,protid)
        stats_plot(entangled,psc,protid)
    
    #Exporting    
    if exporting_cmap3:
        export_cmap3(index,protid,numbering)
        
    if exporting_mat:
        export_mat(index,mat,protid)
        
if exporting_psc:
    export_psc(psclist)

x0000.pdb - 1/1000
Succesfully saved x0000_ _cmap3.csv
x0001.pdb - 2/1000
Succesfully saved x0001_ _cmap3.csv
x0002.pdb - 3/1000
Succesfully saved x0002_ _cmap3.csv
x0003.pdb - 4/1000
Succesfully saved x0003_ _cmap3.csv
x0004.pdb - 5/1000
Succesfully saved x0004_ _cmap3.csv
x0005.pdb - 6/1000
Succesfully saved x0005_ _cmap3.csv
x0006.pdb - 7/1000
Succesfully saved x0006_ _cmap3.csv
x0007.pdb - 8/1000
Succesfully saved x0007_ _cmap3.csv
x0008.pdb - 9/1000
Succesfully saved x0008_ _cmap3.csv
x0009.pdb - 10/1000
Succesfully saved x0009_ _cmap3.csv
x0010.pdb - 11/1000
Succesfully saved x0010_ _cmap3.csv
x0011.pdb - 12/1000
Succesfully saved x0011_ _cmap3.csv
x0012.pdb - 13/1000
Succesfully saved x0012_ _cmap3.csv
x0013.pdb - 14/1000
Succesfully saved x0013_ _cmap3.csv
x0014.pdb - 15/1000
Succesfully saved x0014_ _cmap3.csv
x0015.pdb - 16/1000
Succesfully saved x0015_ _cmap3.csv
x0016.pdb - 17/1000
Succesfully saved x0016_ _cmap3.csv
x0017.pdb - 18/1000
Succesfully saved x0017_ _cmap3.csv
x

In [32]:
cmap_mat = np.zeros([len(numbering),len(numbering)],dtype='int')

for row in index:
        x = row[0]
        y = row[1]
        cmap_mat[x][y] = 1
        #cmap_mat[y][x] = 1
   

cmap_mat.shape

new_index=np.array(np.nonzero(cmap_mat))
new_index=np.transpose(new_index)
new_index

comparison = index == new_index
equal_arrays = comparison.all()
print(equal_arrays)

True


### <center> Secondary structure tool </center>
This function uses the STRIDE tool to calculate the protein's secondary structure. <br> ***NOTE*** STRIDE and DSSP agree in 95,4% of the cases, DSSP tends to assign shorter secondary structures. To use STRIDE files, download them from http://webclu.bio.wzw.tum.de/stride/ and put them in <code>input_files/STRIDE.
</code> 
<br>https://en.wikipedia.org/wiki/STRIDE <br>

It can be used to build a Sec. Struc - Sec. struc contact map, or filter out res-res contacts within a secondary structure.

STRIDE
* H - Alpha-Helix
* B - Isolated Beta-Bridge
* E - Beta Sheet
* b - Isolated Beta-Bridge
* G - 3-10 Helix
* I - Pi helix
* T - Turn
* C - Coil

The following function takes a STRIDE file as input an parses the secondary structure for further use.

In [91]:
structure, sequence = stride_secondary_struc('1bni.txt')

The following function uses the secondary structure to create a secondary structure-secondary structure based cmap (**cmap4**).<br> Note! It creates indices of the nonzero values in the contact map
<br> Keep in mind that this function overwrites certain variables.

In [92]:
cmap4,segment = secondary_struc_cmap(
                                    chain,
                                    sequence,
                                    structure,
                                    cutoff_distance = 4.5,
                                    cutoff_numcontacts = 10,
                                    exclude_neighbour = 3,
                                    ss_elements = ['H','E','b','B','G'])

The following function transforms the indices into a cmap and exports it to <code>results/circuit_diagram</code>

In [ ]:
export_cmap4(cmap4,segment,structure,protid)

This function takes in a res-res contact map and filters out contacts that are within specified secondary structures,<code>filtered_structures</code>.

In [ ]:
cmap5,struc_id = secondary_struc_filter(
                                        index,
                                        structure,
                                        filtered_structures = ['H','E'])

### <center> Circuit analysis </center>

This function uses the circuit theory developed by Anatoly to calculate the amount of circuits within a certain protein. <br>
<code>threshold</code> is the minimum length that will be included.

In [ ]:
threshold = 12
segnums,meanlength,segends = string_pdb(index,numbering,threshold)

export_circuit([protid,segnums,meanlength,segends])

### <center> Multi-chain analysis </center>

Circuit-topology of a whole model is also possible. This uses the same functions but with slightly different options. Keep in mind that it takes longer to compute due to the size of proteins. Import a PDB/CIF file like you normally would using <code>retrieve_chain()</code> and then use the following setting to obtain a contact map of the whole model.

In [ ]:
index,numbering,protid,res_names = get_cmap(
                                            chain,
                                            level = 'model'
                                            cutoff_distance = cutoff_distance,
                                            cutoff_numcontacts = cutoff_numcontacts,        
                                            exclude_neighbour = exclude_neighbour)

The following function is the same, it automatically detects whether it is a model or chain cmap. It does output P,S,C,I,T,L instead of PSC however

In [ ]:
mat,pscitl = get_matrix(index,protid)

The plotting functions also work, There is a different function for the topological matrix plot however.

In [ ]:
circuit_plot(index,protid,numbering)
matrix_plot_model(mat,protid)